In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
image_folder = '/content/drive/MyDrive/Colab Notebooks/3_deskewing/'

**Task 3**

In [1]:
import cv2
import os
from google.colab.patches import cv2_imshow
import numpy as np
from collections import Counter

def deskew_with_double_majority(image_path):
    # Load the image
    original_image = cv2.imread(image_path)
    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion
    kernel = np.ones((25, 25), np.uint8)
    dilation = cv2.dilate(gray, kernel, iterations=2)
    erosion = cv2.erode(dilation, kernel, iterations=1)
    new= erosion*gray


    # Use Canny edge detection
    edges = cv2.Canny(erosion, 50, 150, apertureSize=3)

    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)

    if lines is None or len(lines) < 2:
        raise ValueError("No lines detected for deskewing")

    # Extract angles from detected lines
    angles = [line[0][1] for line in lines]

    # First majority vote: Determine lines
    line_votes = Counter(angles)
    common_lines = line_votes.most_common(2)
    majority_line = common_lines[0][0]

    # Second majority vote: Vote on a common angle among the most important lines
    important_lines = [line for line in lines if line[0][1] == majority_line]
    important_angles = [line[0][1] for line in important_lines]

    angle_votes = Counter(important_angles)
    common_angles = angle_votes.most_common(2)
    majority_angle = common_angles[0][0]

    # Rotate the image to deskew it
    (h, w) = original_image.shape[:2]
    center = (w // 2, h // 2)
    rotation_angle = np.degrees(majority_angle) - 90
    M = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
    deskewed_image = cv2.warpAffine(original_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    deskewed_image_gray = cv2.warpAffine(new, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # Show the original and deskewed images
    print(f"Predicted Angle for {image_path}: {np.degrees(majority_angle):.2f} degrees")
    original_image=cv2.resize(original_image,(512, 512) )
    deskewed_image=cv2.resize(deskewed_image,(512, 512) )
    deskewed_image_gray =cv2.resize(deskewed_image_gray,(512, 512) )
    print("Orignal Image==================================================================")
    cv2_imshow(original_image)
    print("deskewed_image_using Filter==================================================")
    cv2_imshow(deskewed_image_gray)
    print("deskewed_image_e===============================================================")
    cv2_imshow(deskewed_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Replace 'path/to/your/image/folder' with the actual path to your image folder
image_folder = '/content/drive/MyDrive/Colab Notebooks/3_deskewing/'

# Iterate through all files in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, filename)
        deskew_with_double_majority(image_path)
        break



ModuleNotFoundError: No module named 'google.colab'